# Lab 1.3.1: Parallel Reduction

**Module:** 1.3 - CUDA Python & GPU Programming  
**Time:** 3 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand why parallel reduction is fundamental to GPU computing
- [ ] Implement naive parallel reduction and understand its limitations
- [ ] Optimize with shared memory for 10-50x speedup
- [ ] Master warp shuffle operations for ultimate performance
- [ ] Achieve **100x+ speedup** over CPU implementations

---

## 📚 Prerequisites

- Completed: Module 1.2 (Python for AI/ML)
- Knowledge of: Basic NumPy operations, understanding of what a GPU is
- Optional: Read the README.md for this module

---

## 🌍 Real-World Context

**Why does parallel reduction matter?**

When training neural networks, you constantly need to:
- **Sum gradients** across millions of parameters
- **Compute loss** (average over batch)
- **Find max values** (for softmax normalization)
- **Calculate statistics** (mean, variance for batch normalization)

A typical LLM like Llama-70B has **70 billion parameters**. If each reduction operation took even 1 microsecond per parameter on CPU, you'd wait **70 seconds per batch**. With GPU parallel reduction? Under **1 millisecond**.

**Your DGX Spark has 6,144 CUDA cores** ready to reduce arrays in parallel. Let's learn how to use them!

---

## 🧒 ELI5: What is Parallel Reduction?

> **Imagine you're a teacher** with 32 students, and you need to count the total number of candies everyone has.
>
> **The slow way (CPU):** You go to each student one by one: "1 candy... 3 candies... 2 candies..." adding as you go. With 32 students, you need 31 addition steps. With 1 million students? 999,999 steps!
>
> **The fast way (GPU parallel reduction):**
> 1. **Round 1:** Students pair up (16 pairs). Each pair adds their candies together. Now we have 16 numbers.
> 2. **Round 2:** The 16 people pair up (8 pairs). Add again. Now 8 numbers.
> 3. **Round 3:** 8 → 4 numbers
> 4. **Round 4:** 4 → 2 numbers
> 5. **Round 5:** 2 → 1 number (the answer!)
>
> With 32 students, you only need **5 rounds** (log₂(32) = 5), and in each round, all pairs work **simultaneously**!
>
> **In AI terms:** The GPU does thousands of additions at the same time, turning millions of sequential operations into just ~20 parallel rounds. That's how we get 100x+ speedups!

![Parallel Reduction Diagram](https://upload.wikimedia.org/wikipedia/commons/4/4c/Parallel_reduction.png)
*Each level shows one parallel step. All additions in the same level happen simultaneously.*

---

## Part 0: Environment Setup

First, let's verify our CUDA environment and understand our hardware.

In [ ]:
# Standard imports
import numpy as np
import time
from typing import Tuple, Callable
import warnings
warnings.filterwarnings('ignore')

# CUDA imports
from numba import cuda
import numba

# Verify CUDA is available
print("="*60)
print("🔧 CUDA Environment Check")
print("="*60)
print(f"Numba version: {numba.__version__}")
print(f"CUDA available: {cuda.is_available()}")

if cuda.is_available():
    device = cuda.get_current_device()
    print(f"\n📊 GPU Information:")
    print(f"  Device name: {device.name}")
    print(f"  Compute capability: {device.compute_capability}")
    print(f"  Max threads per block: {device.MAX_THREADS_PER_BLOCK}")
    print(f"  Max shared memory per block: {device.MAX_SHARED_MEMORY_PER_BLOCK / 1024:.1f} KB")
    print(f"  Warp size: {device.WARP_SIZE}")
    print(f"  Multiprocessors: {device.MULTIPROCESSOR_COUNT}")
    print("\n✅ Ready for GPU programming!")
else:
    print("\n❌ CUDA not available. Please use NGC container.")
    print("   Run: docker run --gpus all -it nvcr.io/nvidia/pytorch:25.11-py3")

### 🔍 Understanding the Output

On your DGX Spark, you should see:
- **Device name:** Something like "NVIDIA GB10 Superchip" or similar
- **Compute capability:** 10.0 or higher (Blackwell architecture)
- **Max threads per block:** 1024 (standard for modern GPUs)
- **Warp size:** 32 (always 32 on NVIDIA GPUs - this is important!)
- **Multiprocessors:** The number of SMs (Streaming Multiprocessors)

---

## Part 1: CPU Baseline - The Slow Way

Before we make things fast, let's see how slow the CPU approach is. This gives us a baseline to measure our GPU speedups against.

In [ ]:
def cpu_sum_naive(arr: np.ndarray) -> float:
    """Naive CPU sum using Python loop (very slow!)."""
    total = 0.0
    for x in arr:
        total += x
    return total

def cpu_sum_numpy(arr: np.ndarray) -> float:
    """NumPy sum (optimized C, but still sequential)."""
    return np.sum(arr)

# Create test data - 10 million elements
N = 10_000_000
np.random.seed(42)  # For reproducibility
data = np.random.randn(N).astype(np.float32)

print(f"📊 Array size: {N:,} elements ({N * 4 / 1e6:.1f} MB)")
print(f"   Expected sum (numpy): {np.sum(data):.6f}")
print("\n⏱️  Benchmarking CPU approaches...\n")

# Benchmark Python loop (just 1M for speed)
small_data = data[:1_000_000]
start = time.perf_counter()
result_naive = cpu_sum_naive(small_data)
time_naive = time.perf_counter() - start
print(f"Python loop (1M elements): {time_naive*1000:.2f} ms")

# Benchmark NumPy (full 10M)
start = time.perf_counter()
for _ in range(10):  # Average over 10 runs
    result_numpy = cpu_sum_numpy(data)
time_numpy = (time.perf_counter() - start) / 10
print(f"NumPy sum (10M elements): {time_numpy*1000:.2f} ms")

print(f"\n📈 NumPy is ~{time_naive * 10 / time_numpy:.0f}x faster than Python loop")
print("\n💡 Can we beat NumPy with the GPU? Let's find out!")

### 🔍 What Just Happened?

We established our baselines:
- **Python loop:** Embarrassingly slow (~500ms for 1M elements)
- **NumPy:** Much faster (~5ms for 10M) thanks to C optimization

But NumPy still processes elements one at a time (or in small SIMD chunks). The GPU can do **thousands** simultaneously!

---

## Part 2: Your First CUDA Kernel - Naive Reduction

### 🧒 ELI5: What's a Kernel?

> Think of a **kernel** as a recipe that **thousands of chefs** (threads) follow simultaneously. Each chef works on a different ingredient (data element), but they all follow the same instructions.

### Understanding CUDA Thread Hierarchy

```
Grid (all threads)
├── Block 0
│   ├── Thread 0, 1, 2, ... 255 (256 threads)
│   └── (These share "shared memory")
├── Block 1
│   ├── Thread 0, 1, 2, ... 255
│   └── (Their own shared memory)
├── Block 2
│   └── ...
└── ... (many blocks)
```

Key concepts:
- **Thread:** Single worker, has access to its own registers
- **Block:** Group of threads (up to 1024), share fast "shared memory"
- **Grid:** All blocks together, covers the entire problem
- **Warp:** 32 threads that execute in lockstep (hardware unit)

In [ ]:
# Let's understand thread indexing first
@cuda.jit
def print_thread_info(output):
    """Kernel that shows how thread indexing works."""
    # Thread index within block
    tx = cuda.threadIdx.x
    # Block index within grid
    bx = cuda.blockIdx.x
    # Block dimension (number of threads per block)
    bw = cuda.blockDim.x
    
    # Global thread index (unique across entire grid)
    global_idx = bx * bw + tx
    
    # Store info in output array
    if global_idx < output.shape[0]:
        # Pack info: global_idx * 10000 + block_idx * 100 + thread_idx
        output[global_idx] = global_idx * 10000 + bx * 100 + tx

# Launch with 2 blocks of 4 threads each
output = cuda.device_array(8, dtype=np.float32)
print_thread_info[2, 4](output)  # [blocks, threads_per_block]
result = output.copy_to_host()

print("🔍 Thread Indexing Demonstration")
print("="*50)
print(f"Grid config: 2 blocks × 4 threads = 8 total threads\n")
for i, val in enumerate(result):
    global_idx = int(val) // 10000
    block_idx = (int(val) % 10000) // 100
    thread_idx = int(val) % 100
    print(f"Output[{i}]: Global={global_idx}, Block={block_idx}, ThreadInBlock={thread_idx}")

### Now Let's Write Our First Reduction Kernel

**Strategy:** Each block reduces its chunk of data to a single value. Then we sum the block results on CPU.

```
Array: [1, 2, 3, 4, 5, 6, 7, 8]  (8 elements)
       \       /   \       /
        Block 0     Block 1
           ↓           ↓
          10          26
           \         /
            \       /
              36 (final sum, on CPU)
```

In [ ]:
@cuda.jit
def naive_reduce_kernel(data, partial_sums):
    """
    Naive parallel reduction - each block produces one partial sum.
    
    This version has problems (we'll fix them later!):
    - Uses global memory (slow)
    - Has thread divergence issues
    """
    # Get thread and block indices
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    block_size = cuda.blockDim.x
    
    # Global index for this thread
    idx = bx * block_size + tx
    
    # Bounds check
    if idx >= data.size:
        return
    
    # Parallel reduction within block
    # Each iteration, half the threads add their neighbor's value
    stride = 1
    while stride < block_size:
        # Only threads at positions 0, 2*stride, 4*stride, ... do work
        if tx % (2 * stride) == 0:
            if idx + stride < data.size:
                data[idx] += data[idx + stride]
        
        stride *= 2
        cuda.syncthreads()  # Wait for all threads in block
    
    # Thread 0 of each block writes the result
    if tx == 0:
        partial_sums[bx] = data[idx]


def gpu_sum_naive(arr: np.ndarray) -> float:
    """Sum array using naive GPU reduction."""
    n = arr.size
    threads_per_block = 256
    num_blocks = (n + threads_per_block - 1) // threads_per_block
    
    # Copy data to GPU (we modify in place, so need a copy)
    d_data = cuda.to_device(arr.copy())
    d_partial = cuda.device_array(num_blocks, dtype=np.float32)
    
    # Launch kernel
    naive_reduce_kernel[num_blocks, threads_per_block](d_data, d_partial)
    
    # Get partial sums and finish reduction on CPU
    partial_sums = d_partial.copy_to_host()
    return np.sum(partial_sums)


# Test correctness first
test_arr = np.arange(1, 101, dtype=np.float32)  # 1+2+...+100 = 5050
result = gpu_sum_naive(test_arr)
print(f"Test: sum of 1 to 100")
print(f"  Expected: 5050.0")
print(f"  Got:      {result}")
print(f"  Correct:  {'✅ Yes!' if abs(result - 5050.0) < 0.01 else '❌ No'}")

In [ ]:
# Benchmark naive GPU reduction
print("\n⏱️  Benchmarking Naive GPU Reduction...")
print("="*50)

# Warm up GPU
for _ in range(3):
    _ = gpu_sum_naive(data)
cuda.synchronize()  # Ensure GPU is done

# Benchmark
start = time.perf_counter()
for _ in range(10):
    result_gpu = gpu_sum_naive(data)
    cuda.synchronize()
time_naive_gpu = (time.perf_counter() - start) / 10

print(f"Array size: {N:,} elements")
print(f"NumPy time:     {time_numpy*1000:.2f} ms")
print(f"Naive GPU time: {time_naive_gpu*1000:.2f} ms")
print(f"Speedup: {time_numpy/time_naive_gpu:.1f}x {'faster' if time_naive_gpu < time_numpy else 'slower'} than NumPy")

# Check result accuracy
print(f"\n🎯 Accuracy check:")
print(f"   NumPy result: {result_numpy:.6f}")
print(f"   GPU result:   {result_gpu:.6f}")
print(f"   Difference:   {abs(result_numpy - result_gpu):.6f}")

### 🔍 What Just Happened?

Hmm, our naive GPU version might actually be **slower** than NumPy! Why?

**Problems with our naive implementation:**

1. **Global Memory Access:** Reading/writing to `data[]` goes to slow global memory (273 GB/s sounds fast, but shared memory is 10x faster)

2. **Thread Divergence:** The `if tx % (2 * stride) == 0` condition means threads in the same warp take different paths. GPUs execute all 32 threads in a warp together - divergent threads just wait!

3. **Memory Copy Overhead:** We're copying data to GPU and back for each call

Let's fix these issues!

---

## Part 3: Optimized Reduction with Shared Memory

### 🧒 ELI5: What's Shared Memory?

> Imagine a classroom (block) where students (threads) need to share answers.
>
> **Global memory** is like shouting across a huge stadium - everyone can hear, but it's slow and crowded.
>
> **Shared memory** is like a small whiteboard in the classroom - only your classmates can see it, but it's right there, super fast to read and write!
>
> **The trick:** Load data from the stadium (global) to the whiteboard (shared), do all your work on the whiteboard, then write the final answer back to the stadium.

### Memory Hierarchy Speed Comparison

| Memory Type | Speed | Size per SM | Access Scope |
|-------------|-------|-------------|-------------|
| Registers | ~1 cycle | 256 KB | Per thread |
| Shared Memory | ~5 cycles | 164 KB | Per block |
| L1 Cache | ~30 cycles | 128 KB | Per SM |
| L2 Cache | ~100 cycles | 62 MB | All SMs |
| Global Memory | ~500 cycles | 128 GB | All threads |

In [ ]:
# Shared memory size must be known at compile time
THREADS_PER_BLOCK = 256

@cuda.jit
def shared_memory_reduce_kernel(data, partial_sums):
    """
    Optimized reduction using shared memory.
    
    Improvements over naive:
    1. Uses fast shared memory instead of global memory
    2. Sequential addressing to avoid bank conflicts
    3. Better thread utilization pattern
    """
    # Allocate shared memory for this block
    # This is ~100x faster than global memory!
    sdata = cuda.shared.array(shape=(THREADS_PER_BLOCK,), dtype=numba.float32)
    
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    block_size = cuda.blockDim.x
    idx = bx * block_size + tx
    
    # Step 1: Load from global memory to shared memory
    if idx < data.size:
        sdata[tx] = data[idx]
    else:
        sdata[tx] = 0.0  # Padding for out-of-bounds threads
    
    # CRITICAL: Wait for all threads to finish loading
    cuda.syncthreads()
    
    # Step 2: Reduction in shared memory
    # Use sequential addressing to avoid bank conflicts
    stride = block_size // 2
    while stride > 0:
        if tx < stride:
            sdata[tx] += sdata[tx + stride]
        cuda.syncthreads()
        stride //= 2
    
    # Step 3: Thread 0 writes result to global memory
    if tx == 0:
        partial_sums[bx] = sdata[0]


def gpu_sum_shared(arr: np.ndarray) -> float:
    """Sum array using shared memory reduction."""
    n = arr.size
    num_blocks = (n + THREADS_PER_BLOCK - 1) // THREADS_PER_BLOCK
    
    d_data = cuda.to_device(arr)
    d_partial = cuda.device_array(num_blocks, dtype=np.float32)
    
    shared_memory_reduce_kernel[num_blocks, THREADS_PER_BLOCK](d_data, d_partial)
    
    partial_sums = d_partial.copy_to_host()
    return np.sum(partial_sums)


# Test correctness
result = gpu_sum_shared(test_arr)
print(f"Shared Memory Reduction Test:")
print(f"  Expected: 5050.0")
print(f"  Got:      {result}")
print(f"  Correct:  {'✅ Yes!' if abs(result - 5050.0) < 0.01 else '❌ No'}")

In [ ]:
# Benchmark shared memory version
print("\n⏱️  Benchmarking Shared Memory Reduction...")
print("="*50)

# Warm up
for _ in range(3):
    _ = gpu_sum_shared(data)
cuda.synchronize()

# Benchmark
start = time.perf_counter()
for _ in range(10):
    result_shared = gpu_sum_shared(data)
    cuda.synchronize()
time_shared = (time.perf_counter() - start) / 10

print(f"Array size: {N:,} elements")
print(f"")
print(f"NumPy time:          {time_numpy*1000:.3f} ms")
print(f"Naive GPU time:      {time_naive_gpu*1000:.3f} ms")
print(f"Shared Memory time:  {time_shared*1000:.3f} ms")
print(f"")
print(f"📈 Shared vs Naive:  {time_naive_gpu/time_shared:.1f}x faster")
print(f"📈 Shared vs NumPy:  {time_numpy/time_shared:.1f}x faster")

### 🔍 Why Is Shared Memory Faster?

**Before (Naive):**
```
Thread 0: Read global[0], Read global[1], Write global[0]  ← 3 slow ops
Thread 0: Read global[0], Read global[2], Write global[0]  ← 3 slow ops
...repeat log₂(N) times...
```

**After (Shared Memory):**
```
All threads: Read global[tx] → shared[tx]  ← 1 slow op, done once!
Thread 0: Read shared[0], Read shared[128], Write shared[0]  ← 3 fast ops
Thread 0: Read shared[0], Read shared[64], Write shared[0]   ← 3 fast ops
...repeat log₂(BLOCK_SIZE) times using fast shared memory...
Thread 0: Write global[bx] = shared[0]  ← 1 slow op
```

**The key insight:** We minimized global memory access and did all the heavy computation in fast shared memory!

---

## Part 4: Ultimate Optimization with Warp Shuffle

### 🧒 ELI5: What's a Warp Shuffle?

> Remember the 32 students who always work together (the warp)? Usually, if student 0 wants to see student 16's answer, they write it on the whiteboard and student 0 reads it. Two operations!
>
> **Warp shuffle** is like telepathy between the 32 students. Student 0 can directly peek at student 16's paper without any writing. It's instantaneous!
>
> This is a special hardware feature where threads in the same warp can directly share register values without using any memory at all.

### Warp-Level Primitives

CUDA provides special "warp shuffle" instructions:
- `__shfl_down_sync()`: Get value from thread with higher index
- `__shfl_up_sync()`: Get value from thread with lower index  
- `__shfl_xor_sync()`: Get value from thread at XOR distance

In Numba, we use `cuda.shfl_down_sync()` and similar.

In [ ]:
WARP_SIZE = 32

@cuda.jit(device=True)
def warp_reduce(val):
    """
    Reduce within a warp using shuffle instructions.
    
    This is a device function - called from kernels, not from host.
    Uses warp shuffle for ultimate performance (no memory access!).
    
    After this, thread 0 of the warp has the sum of all 32 values.
    """
    # All 32 threads participate (mask = 0xffffffff)
    mask = 0xffffffff
    
    # Shuffle down and add in tree pattern
    # Round 1: threads 0-15 get values from threads 16-31
    val += cuda.shfl_down_sync(mask, val, 16)
    # Round 2: threads 0-7 get values from threads 8-15
    val += cuda.shfl_down_sync(mask, val, 8)
    # Round 3: threads 0-3 get values from threads 4-7
    val += cuda.shfl_down_sync(mask, val, 4)
    # Round 4: threads 0-1 get values from threads 2-3
    val += cuda.shfl_down_sync(mask, val, 2)
    # Round 5: thread 0 gets value from thread 1
    val += cuda.shfl_down_sync(mask, val, 1)
    
    return val


@cuda.jit
def warp_shuffle_reduce_kernel(data, partial_sums):
    """
    Ultimate reduction using warp shuffle + shared memory hybrid.
    
    Strategy:
    1. Each thread loads data from global memory
    2. Warp-level reduction (no shared memory needed!)
    3. Store warp results in shared memory
    4. Final reduction of warp results
    """
    # Shared memory only for warp results (much smaller!)
    # With 256 threads, we have 8 warps, so we need 8 slots
    warp_sums = cuda.shared.array(shape=(THREADS_PER_BLOCK // WARP_SIZE,), dtype=numba.float32)
    
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    block_size = cuda.blockDim.x
    idx = bx * block_size + tx
    
    # Which warp am I in? Which lane within the warp?
    warp_id = tx // WARP_SIZE      # 0, 1, 2, ... 7 for 256 threads
    lane_id = tx % WARP_SIZE       # 0, 1, 2, ... 31 within warp
    
    # Step 1: Load data
    val = data[idx] if idx < data.size else 0.0
    
    # Step 2: Warp-level reduction (no syncthreads needed!)
    val = warp_reduce(val)
    
    # Step 3: Lane 0 of each warp stores to shared memory
    if lane_id == 0:
        warp_sums[warp_id] = val
    
    cuda.syncthreads()  # Wait for all warps to finish
    
    # Step 4: First warp reduces all warp sums
    num_warps = block_size // WARP_SIZE
    if warp_id == 0:
        val = warp_sums[lane_id] if lane_id < num_warps else 0.0
        val = warp_reduce(val)
        
        if lane_id == 0:
            partial_sums[bx] = val


def gpu_sum_warp_shuffle(arr: np.ndarray) -> float:
    """Sum array using warp shuffle reduction."""
    n = arr.size
    num_blocks = (n + THREADS_PER_BLOCK - 1) // THREADS_PER_BLOCK
    
    d_data = cuda.to_device(arr)
    d_partial = cuda.device_array(num_blocks, dtype=np.float32)
    
    warp_shuffle_reduce_kernel[num_blocks, THREADS_PER_BLOCK](d_data, d_partial)
    
    partial_sums = d_partial.copy_to_host()
    return np.sum(partial_sums)


# Test correctness
result = gpu_sum_warp_shuffle(test_arr)
print(f"Warp Shuffle Reduction Test:")
print(f"  Expected: 5050.0")
print(f"  Got:      {result}")
print(f"  Correct:  {'✅ Yes!' if abs(result - 5050.0) < 0.01 else '❌ No'}")

In [ ]:
# Benchmark warp shuffle version
print("\n⏱️  Benchmarking Warp Shuffle Reduction...")
print("="*50)

# Warm up
for _ in range(3):
    _ = gpu_sum_warp_shuffle(data)
cuda.synchronize()

# Benchmark
start = time.perf_counter()
for _ in range(10):
    result_shuffle = gpu_sum_warp_shuffle(data)
    cuda.synchronize()
time_shuffle = (time.perf_counter() - start) / 10

print(f"Array size: {N:,} elements\n")
print(f"{'Method':<25} {'Time (ms)':<12} {'vs NumPy':<12}")
print("-" * 50)
print(f"{'NumPy':<25} {time_numpy*1000:<12.3f} {'1.0x':<12}")
print(f"{'Naive GPU':<25} {time_naive_gpu*1000:<12.3f} {f'{time_numpy/time_naive_gpu:.1f}x':<12}")
print(f"{'Shared Memory':<25} {time_shared*1000:<12.3f} {f'{time_numpy/time_shared:.1f}x':<12}")
print(f"{'Warp Shuffle':<25} {time_shuffle*1000:<12.3f} {f'{time_numpy/time_shuffle:.1f}x':<12}")

print(f"\n🎉 Best speedup: {time_numpy/time_shuffle:.1f}x faster than NumPy!")

---

## Part 5: Complete Reduction with Multi-Level Kernel

Our previous versions still do the final sum on CPU. For maximum performance (especially with very large arrays), we should do the entire reduction on GPU.

**Strategy:** Launch kernels repeatedly until we're down to 1 element.

```
10,000,000 elements
    ↓ Kernel 1: 39,063 blocks → 39,063 partial sums
39,063 elements
    ↓ Kernel 2: 153 blocks → 153 partial sums
153 elements
    ↓ Kernel 3: 1 block → 1 sum
Final answer!
```

In [ ]:
def gpu_sum_complete(arr: np.ndarray) -> float:
    """
    Complete GPU reduction with multi-level kernels.
    
    This avoids any CPU reduction - everything happens on GPU.
    """
    n = arr.size
    d_data = cuda.to_device(arr)
    
    while n > 1:
        num_blocks = (n + THREADS_PER_BLOCK - 1) // THREADS_PER_BLOCK
        d_partial = cuda.device_array(num_blocks, dtype=np.float32)
        
        warp_shuffle_reduce_kernel[num_blocks, THREADS_PER_BLOCK](d_data, d_partial)
        
        # Prepare for next iteration
        d_data = d_partial
        n = num_blocks
    
    # Only copy 1 element back
    return d_data.copy_to_host()[0]


# Test
result = gpu_sum_complete(test_arr)
print(f"Complete GPU Reduction Test:")
print(f"  Expected: 5050.0")
print(f"  Got:      {result}")
print(f"  Correct:  {'✅ Yes!' if abs(result - 5050.0) < 0.01 else '❌ No'}")

In [ ]:
# Final comprehensive benchmark
print("\n" + "="*60)
print("📊 FINAL BENCHMARK: Parallel Reduction Performance")
print("="*60)

# Test at multiple sizes
sizes = [100_000, 1_000_000, 10_000_000, 50_000_000]
methods = [
    ('NumPy', lambda x: np.sum(x)),
    ('Shared Memory', gpu_sum_shared),
    ('Warp Shuffle', gpu_sum_warp_shuffle),
    ('Complete GPU', gpu_sum_complete),
]

print(f"\n{'Size':>12} | " + " | ".join(f"{name:>15}" for name, _ in methods))
print("-" * (14 + 18 * len(methods)))

for size in sizes:
    test_data = np.random.randn(size).astype(np.float32)
    times = []
    
    for name, func in methods:
        # Warm up
        for _ in range(2):
            _ = func(test_data)
        if name != 'NumPy':
            cuda.synchronize()
        
        # Benchmark
        start = time.perf_counter()
        for _ in range(5):
            _ = func(test_data)
        if name != 'NumPy':
            cuda.synchronize()
        elapsed = (time.perf_counter() - start) / 5
        times.append(elapsed)
    
    # Print with speedups relative to NumPy
    numpy_time = times[0]
    row = f"{size:>12,} | "
    for i, (name, _) in enumerate(methods):
        if i == 0:
            row += f"{times[i]*1000:>12.2f} ms | "
        else:
            speedup = numpy_time / times[i]
            row += f"{times[i]*1000:>8.2f} ({speedup:>3.0f}x) | "
    print(row)

print("\n💡 Numbers in parentheses show speedup vs NumPy")

---

## ⚠️ Common Mistakes

### Mistake 1: Forgetting `cuda.syncthreads()`

In [ ]:
# ❌ WRONG: Missing synchronization
@cuda.jit
def broken_reduce(data, output):
    sdata = cuda.shared.array(shape=(256,), dtype=numba.float32)
    tx = cuda.threadIdx.x
    sdata[tx] = data[cuda.blockIdx.x * 256 + tx]
    # Missing: cuda.syncthreads()  ← BUG!
    if tx < 128:
        sdata[tx] += sdata[tx + 128]  # Reading uninitialized values!

# ✅ CORRECT: Synchronize before reading
@cuda.jit
def correct_reduce(data, output):
    sdata = cuda.shared.array(shape=(256,), dtype=numba.float32)
    tx = cuda.threadIdx.x
    sdata[tx] = data[cuda.blockIdx.x * 256 + tx]
    cuda.syncthreads()  # ← All threads wait here
    if tx < 128:
        sdata[tx] += sdata[tx + 128]  # Now safe!

print("💡 Always use cuda.syncthreads() after writing to shared memory!")

### Mistake 2: Conditional `syncthreads()` - Deadlock!

In [ ]:
# ❌ WRONG: syncthreads inside conditional
@cuda.jit
def deadlock_kernel(data):
    tx = cuda.threadIdx.x
    if tx < 128:  # Only half the threads!
        # ... do work ...
        cuda.syncthreads()  # ← DEADLOCK! Other threads never reach this!

# ✅ CORRECT: All threads reach syncthreads
@cuda.jit
def working_kernel(data):
    tx = cuda.threadIdx.x
    if tx < 128:
        pass  # ... do work ...
    cuda.syncthreads()  # ← All threads reach this, even if they didn't work

print("💡 cuda.syncthreads() must be reached by ALL threads in the block!")
print("   It's a barrier - threads wait until everyone arrives.")

### Mistake 3: Not Checking Array Bounds

In [ ]:
# ❌ WRONG: No bounds check
@cuda.jit
def oob_kernel(data, output):
    idx = cuda.grid(1)
    output[idx] = data[idx] * 2  # ← Crashes if idx >= data.size!

# ✅ CORRECT: Check bounds
@cuda.jit
def safe_kernel(data, output):
    idx = cuda.grid(1)
    if idx < data.size:  # ← Safety check!
        output[idx] = data[idx] * 2

print("💡 Always check: if idx < data.size before accessing arrays!")
print("   GPU launches threads in multiples of block size.")
print("   Example: 1000 elements, 256 threads/block → 1024 threads launched!")

### Mistake 4: Not Synchronizing After Kernel Launch

In [ ]:
# ❌ WRONG: Timing without synchronization
def bad_benchmark():
    d_data = cuda.to_device(np.ones(1000000, dtype=np.float32))
    start = time.perf_counter()
    some_kernel[blocks, threads](d_data)  # This is ASYNCHRONOUS!
    elapsed = time.perf_counter() - start  # ← Measures launch time, not execution!
    return elapsed

# ✅ CORRECT: Synchronize before measuring
def good_benchmark():
    d_data = cuda.to_device(np.ones(1000000, dtype=np.float32))
    start = time.perf_counter()
    some_kernel[blocks, threads](d_data)
    cuda.synchronize()  # ← Wait for GPU to finish!
    elapsed = time.perf_counter() - start
    return elapsed

print("💡 Kernel launches are ASYNCHRONOUS!")
print("   Use cuda.synchronize() before timing or reading results.")

---

## ✋ Try It Yourself: Implement Parallel Max

**Challenge:** Modify the reduction kernel to find the **maximum** value instead of the sum.

This is the same pattern used in:
- Softmax normalization (find max for numerical stability)
- Finding the predicted class (argmax)
- Computing attention scores

In [ ]:
# TODO: Implement parallel max reduction
# Hint: Replace addition with max comparison

@cuda.jit(device=True)
def warp_max(val):
    """Find max within a warp using shuffle instructions."""
    mask = 0xffffffff
    
    # TODO: Replace += with max()
    # Hint: Use max(val, cuda.shfl_down_sync(...))
    # YOUR CODE HERE
    pass
    
    return val


@cuda.jit
def parallel_max_kernel(data, partial_maxes):
    """Find max value using parallel reduction."""
    # TODO: Implement this kernel
    # Hint: It's almost identical to warp_shuffle_reduce_kernel
    # Just replace addition with max()
    # YOUR CODE HERE
    pass


def gpu_max(arr: np.ndarray) -> float:
    """Find max value using GPU."""
    # TODO: Implement this function
    # YOUR CODE HERE
    pass


# Test your implementation
test_data = np.random.randn(1_000_000).astype(np.float32)
expected_max = np.max(test_data)

# Uncomment when implemented:
# result = gpu_max(test_data)
# print(f"Expected max: {expected_max}")
# print(f"GPU max:      {result}")
# print(f"Correct: {'✅' if abs(result - expected_max) < 1e-5 else '❌'}")

<details>
<summary>💡 Hint 1</summary>

The only change needed in `warp_max` is:
```python
val = max(val, cuda.shfl_down_sync(mask, val, offset))
```
Instead of:
```python
val += cuda.shfl_down_sync(mask, val, offset)
```
</details>

<details>
<summary>💡 Hint 2</summary>

For the kernel, initialize with negative infinity for proper max behavior:
```python
val = data[idx] if idx < data.size else -np.inf
```
</details>

<details>
<summary>💡 Hint 3</summary>

When combining warp results, also use max:
```python
if lane_id == 0:
    warp_maxes[warp_id] = val
# ...
val = warp_maxes[lane_id] if lane_id < num_warps else -np.inf
val = warp_max(val)
```
</details>

---

## 🎉 Checkpoint

Congratulations! You've learned:

- ✅ **Why parallel reduction matters** - It's the foundation of neural network training
- ✅ **GPU thread hierarchy** - Threads, blocks, warps, and how they map to hardware
- ✅ **Shared memory optimization** - 100x faster than global memory
- ✅ **Warp shuffle operations** - Direct thread-to-thread communication
- ✅ **Common pitfalls** - syncthreads, bounds checking, synchronization

You achieved **100x+ speedup** over CPU implementations!

---

## 🚀 Challenge (Optional)

**Advanced Challenge: Implement Double-Precision Reduction**

Floating-point addition is not associative! The order of operations can change the result slightly.

```python
# Example: Adding 1e8 + 1e-8 + 1e-8 + 1e-8 + ...
# Order 1: ((1e8 + 1e-8) + 1e-8) + ... = 1e8 (small values lost!)
# Order 2: (1e-8 + 1e-8 + ...) + 1e8 = 1e8 + small_sum (more accurate)
```

**Your challenge:**
1. Implement reduction using `float64` (double precision)
2. Compare accuracy vs `float32` version
3. Implement **Kahan summation** for even better accuracy

*Kahan summation tracks a "compensation" term to recover lost precision.*

---

## 📖 Further Reading

- [NVIDIA Parallel Reduction PDF](https://developer.download.nvidia.com/assets/cuda/files/reduction.pdf) - The classic optimization guide
- [Numba CUDA Documentation](https://numba.readthedocs.io/en/stable/cuda/index.html)
- [Warp Shuffle Functions](https://developer.nvidia.com/blog/using-cuda-warp-level-primitives/)
- [Understanding GPU Architecture](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation)

---

## 🧹 Cleanup

In [ ]:
# Clear GPU memory
import gc

# Delete large arrays
del data, test_data
if 'small_data' in dir():
    del small_data

# Force garbage collection
gc.collect()

# Clear CUDA context
cuda.current_context().reset()

print("✅ GPU memory cleared!")
print("\n➡️ Ready for Lab 1.3.2: Matrix Multiplication")